In [88]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dlib

In [89]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [90]:
for i in range(100):
    globals()['land'+str(i)] = pd.read_csv('./dlib(denorm)/train/%s.csv'%i)
    globals()['land'+str(i)] = np.array(globals()['land'+str(i)])
    
    tr_x = globals()['land'+str(i)][:, 0]
    tr_y = globals()['land'+str(i)][:, 1]
    
    tr_cx = ((min(tr_x)+max(tr_x)) / 2)        #dlib_rect 중점
    tr_cy = ((min(tr_y)+max(tr_y)) / 2)
    
    tr_h = max(tr_y)-min(tr_y)    #종축 길이
    
    tr_x -= tr_cx
    tr_y -= tr_cy
    
    norm_trx = (tr_x - min(tr_x)) / tr_h
    norm_try = (tr_y - min(tr_y)) / (tr_h *1.5)
    
    globals()['train_norm'+str(i)] = np.column_stack((norm_trx, norm_try))

In [91]:
## eye, nose, mouth, jaws
for i in  range(100):
    globals()['norm_eye'+str(i)] = globals()['train_norm'+str(i)][36:48].flatten()
    globals()['norm_nose'+str(i)] = globals()['train_norm'+str(i)][27:36].flatten()
    globals()['norm_mouth'+str(i)] = globals()['train_norm'+str(i)][48:].flatten()
    globals()['norm_jaws'+str(i)] = globals()['train_norm'+str(i)][:17].flatten()

In [92]:
face_path = './testimg/img.png'
face_origin = cv2.imread(face_path)
face_copy = face_origin.copy()
face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
detector = dlib.get_frontal_face_detector()
dlib_rects = detector(face_rgb, 1)
    
img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
img = []
    
for dlib_rect in dlib_rects:
    points = predictor(face_rgb, dlib_rect)
    list_points = list(map(lambda p: [p.x, p.y], points.parts()))
    img.append(list_points)
    img = np.array(img)
    
land = np.zeros((68,2))
    
for j in range(68):
    land[j][0] = img[0][j][0]
    land[j][1] = img[0][j][1]
        
x = land[:,0]
y = land[:,1]
    
x -= ((min(x) + max(x)) / 2)
y -= ((min(y) + max(y)) / 2)
    
norm_x = (x - min(x)) / (max(y)-min(y))
norm_y = (y - min(y)) / ((max(y)-min(y)) * 1.5)
    
norm = np.column_stack((norm_x, norm_y))
norm_eye = norm[36:48].flatten()
norm_nose = norm[27:36].flatten()
norm_mouth = norm[48:].flatten()
norm_jaws = norm[:17].flatten()

In [93]:
try_eye = pd.read_csv('./dataset/train/train.csv')
try_eye = try_eye.iloc[:, 16:22]
try_nose = pd.read_csv('./dataset/train/train.csv')
try_nose = try_nose.iloc[:, 22:30]
try_mouth = pd.read_csv('./dataset/train/train.csv')
try_mouth = try_mouth.iloc[:, 30:]
try_jaws = pd.read_csv('./dataset/train/train.csv')
try_jaws = try_jaws.iloc[:, 5:9]

In [84]:
# tey_eye = pd.read_csv('./dataset/test/test.csv')
# tey_eye = tey_eye.iloc[:, 16:22]
# tey_nose = pd.read_csv('./dataset/test/test.csv')
# tey_nose = tey_nose.iloc[:, 22:30]
# tey_mouth = pd.read_csv('./dataset/test/test.csv')
# tey_mouth = tey_mouth.iloc[:, 30:]
# tey_jaws = pd.read_csv('./dataset/test/test.csv')
# tey_jaws = tey_jaws.iloc[:, 5:9]

In [94]:
train_x = np.zeros((100, len(norm_eye0)))   # eye
for i in range(100):
    for j in range(len(norm_eye0)):
        train_x[i][j] = globals()['norm_eye'+str(i)][j]   
        
test_x = np.zeros((1, len(norm_eye)))
for j in range(len(norm_eye)):
    test_x[0][j] = norm_eye[j]
    
model = LinearRegression()
model.fit(train_x,try_eye)
y_pred = model.predict(test_x)
prediction = pd.DataFrame(y_pred)

In [96]:
train_x = np.zeros((100, len(norm_nose0)))   # nose
for i in range(100):
    for j in range(len(norm_nose0)):
        train_x[i][j] = globals()['norm_nose'+str(i)][j]   
        
test_x = np.zeros((1, len(norm_nose)))
for j in range(len(norm_nose)):
    test_x[0][j] = norm_nose[j]
    
model = LinearRegression()
model.fit(train_x,try_nose)
y_pred = model.predict(test_x)
prediction = pd.DataFrame(y_pred)

In [98]:
train_x = np.zeros((100, len(norm_mouth0)))   # mouth
for i in range(100):
    for j in range(len(norm_mouth0)):
        train_x[i][j] = globals()['norm_mouth'+str(i)][j]   
        
test_x = np.zeros((1, len(norm_mouth)))
for j in range(len(norm_mouth)):
    test_x[0][j] = norm_mouth[j]
    
model = LinearRegression()
model.fit(train_x,try_mouth)
y_pred = model.predict(test_x)
prediction = pd.DataFrame(y_pred)

In [100]:
train_x = np.zeros((100, len(norm_jaws0)))   # jaws
for i in range(100):
    for j in range(len(norm_jaws0)):
        train_x[i][j] = globals()['norm_jaws'+str(i)][j]   
        
test_x = np.zeros((1, len(norm_jaws)))
for j in range(len(norm_jaws)):
    test_x[0][j] = norm_jaws[j]
    
model = LinearRegression()
model.fit(train_x,try_jaws)
y_pred = model.predict(test_x)
prediction = pd.DataFrame(y_pred)

In [ ]:
### im.jpg prediction

In [95]:
prediction   #eye

,0,1,2,3,4,5
0,0.315841,0.67116,0.446887,0.71298,0.196987,0.579441


In [97]:
prediction   #nose

,0,1,2,3,4,5,6,7
0,0.662556,-0.009613,0.293989,0.714866,0.726981,0.863599,0.667635,0.795339


In [99]:
prediction   #mouth

,0,1,2,3
0,0.804053,0.188137,0.472745,0.702762


In [101]:
prediction #jaws

,0,1,2,3
0,0.526116,0.603691,0.194428,0.453217


In [70]:
# tey_eye

In [103]:
# mae = mean_absolute_error(tey_nose, y_pred)
# mae